## مثال و کد پارتیشن با METIS

In [2]:
!pip install networkx
!pip install pymetis

In [3]:
import networkx as nx
import pymetis

 # گراف نمونه
G = nx.karate_club_graph()
n_parts = 4

# ساخت adjacency list مناسب pymetis
adj = [list(G.neighbors(i)) for i in G.nodes()]
edgecuts, parts = pymetis.part_graph(n_parts, adjacency=adj)

print("Edge cuts:", edgecuts)
for node, part in zip(G.nodes(), parts):
    print(f"Node {node} -> Partition {part}")

Edge cuts: 27
Node 0 -> Partition 2
Node 1 -> Partition 2
Node 2 -> Partition 2
Node 3 -> Partition 2
Node 4 -> Partition 3
Node 5 -> Partition 3
Node 6 -> Partition 3
Node 7 -> Partition 2
Node 8 -> Partition 1
Node 9 -> Partition 3
Node 10 -> Partition 3
Node 11 -> Partition 3
Node 12 -> Partition 3
Node 13 -> Partition 2
Node 14 -> Partition 1
Node 15 -> Partition 1
Node 16 -> Partition 3
Node 17 -> Partition 2
Node 18 -> Partition 1
Node 19 -> Partition 2
Node 20 -> Partition 1
Node 21 -> Partition 2
Node 22 -> Partition 1
Node 23 -> Partition 0
Node 24 -> Partition 0
Node 25 -> Partition 0
Node 26 -> Partition 0
Node 27 -> Partition 0
Node 28 -> Partition 0
Node 29 -> Partition 0
Node 30 -> Partition 1
Node 31 -> Partition 0
Node 32 -> Partition 1
Node 33 -> Partition 1


### کد ساده DropEdge

In [6]:
!pip install torch

In [8]:
import torch

# Example edge_index (2 × num_edges)
edge_index = torch.tensor([
    [0, 1, 2, 3],
    [1, 0, 3, 2]
], dtype=torch.long)

def drop_edge(edge_index, drop_rate=0.2):
    m = edge_index.size(1)
    mask = (torch.rand(m) > drop_rate)
    return edge_index[:, mask]

new_edge_index = drop_edge(edge_index, 0.2)
print(new_edge_index)


tensor([[0, 1, 2, 3],
        [1, 0, 3, 2]])


## مثال ساده از Aggregation Caching

In [9]:
cache = {}

def aggregate_with_cache(node, adj, X):
    neigh = tuple(sorted(adj[node].tolist()))

    if neigh in cache:
        return cache[neigh]

    aggr = X[adj[node]].mean(dim=0)
    cache[neigh] = aggr

    return aggr

## مثال Quantization ساده (8bit)

In [10]:
import torch

def quantize_8bit(x):
    x_min = x.min()
    x_max = x.max()
    scale = (x_max - x_min) / 255.0

    q = torch.round((x - x_min) / scale).clamp(0, 255).to(torch.uint8)
    return q, x_min, scale

def dequantize_8bit(q, x_min, scale):
    return q.float() * scale + x_min

## مثال: درجه‌محور (Hub Sorting)

In [11]:
import networkx as nx

# Create a sample graph with edges
edges = [
    (0, 1),
    (0, 2),
    (1, 2),
    (1, 3),
    (2, 3),
    (3, 4)
]

G = nx.Graph()
G.add_edges_from(edges)

# Calculate the degree of each node
deg = dict(G.degree())

# Sort nodes by degree (highest to lowest)
sorted_nodes = sorted(G.nodes(), key=lambda n: deg[n], reverse=True)

# Create a new mapping for nodes
mapping = {old: new for new, old in enumerate(sorted_nodes)}

# Apply the mapping to the graph
G_reordered = nx.relabel_nodes(G, mapping)

print("Node mapping:", mapping)
print("Edges of reordered graph:", list(G_reordered.edges()))


Node mapping: {1: 0, 2: 1, 3: 2, 0: 3, 4: 4}
Edges of reordered graph: [(3, 0), (3, 1), (0, 1), (0, 2), (1, 2), (2, 4)]


## مثال ساده: ساخت Shard بر اساس ID گرهٔ مبدا

In [12]:
import torch

def make_shards(edge_index, num_shards=4):
    N = int(edge_index.max().item()) + 1
    shard_size = (N + num_shards - 1) // num_shards

    shards = []
    for i in range(num_shards):
        lo = i * shard_size
        hi = (i + 1) * shard_size
        mask = (edge_index[0] >= lo) & (edge_index[0] < hi)
        shards.append(edge_index[:, mask])

    return shards


### - PageRank

In [13]:
import networkx as nx

deg = dict(nx.degree(G))
clustering = nx.clustering(G)
pagerank = nx.pagerank(G)

### محاسبهٔ eigenvectorهای کوچک:

In [14]:
from scipy.sparse.linalg import eigsh
import networkx as nx
import numpy as np

# normalized Laplacian
L = nx.normalized_laplacian_matrix(G)

# k must be less than number of nodes
k = min(8, G.number_of_nodes() - 1)
vals, vecs = eigsh(L, k=k)
positional_emb = vecs

# print results
print("Eigenvalues:", vals)
print("Positional embeddings (first few rows):\n", positional_emb)

Eigenvalues: [0.56574145 1.33333333 1.33333333 1.76759188]
Positional embeddings (first few rows):
 [[-0.46140187 -0.70703179 -0.01029764 -0.34704704]
 [-0.24539932  0.27834688  0.71123579  0.32626039]
 [-0.24539932  0.29894216 -0.7028278   0.32626039]
 [ 0.49079864  0.28864452  0.00420399 -0.65252078]
 [ 0.65252078 -0.49994698 -0.00728153  0.49079864]]


# ۳) Random-walk Features

In [15]:
!pip install node2vec

In [16]:
from node2vec import Node2Vec

node2vec = Node2Vec(G, dimensions=16, walk_length=20, num_walks=200)
model = node2vec.fit()
emb = model.wv

Computing transition probabilities:   0%|          | 0/5 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 200/200 [00:00<00:00, 1288.13it/s]


# ۴) Edge Features

In [17]:
import networkx as nx

for u, v in G.edges():
    G[u][v]["triangles"] = len(list(nx.common_neighbors(G, u, v)))